In [2]:
import os
import json
import pandas as pd
import traceback

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
key=os.getenv("GOOGLE_API_KEY")

In [6]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    api_key=key,
    temperature=0.2,
)

In [7]:

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import PyPDF2


In [8]:

RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [9]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [10]:
quiz_generation_prompt = PromptTemplate(
    input_variables = ["text","number" ,"subject", "tone", "response_json"],
    template=TEMPLATE
)

In [11]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz" , verbose=True) 

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_26220\242460186.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz" , verbose=True)


In [12]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [13]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [14]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [15]:
generate_evaluate_chain = SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text","number","subject","tone","response_json"], output_variables=["quiz","review"],verbose=True,)

In [16]:
file_path =r"D:\Users\Lenovo\Downloads\mcqgen\data.txt"

In [17]:
file_path

'D:\\Users\\Lenovo\\Downloads\\mcqgen\\data.txt'

In [18]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [19]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems contributed 

In [ ]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [25]:
NUMBER = 5
SUBJECT = "MACHINE LEARNING"
TONE = "simple"

In [37]:
from langchain.callbacks import StdOutCallbackHandler

handler = StdOutCallbackHandler()

response = generate_evaluate_chain({
    "text": TEXT,
    "number": NUMBER,
    "subject": SUBJECT,
    "tone": TONE,
    "response_json": json.dumps(RESPONSE_JSON),
} , callbacks=[handler]) 



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used b

In [41]:
import tiktoken

def estimate_tokens(text):
    encoding = tiktoken.get_encoding("cl100k_base")
    return len(encoding.encode(text))

input_text = TEXT + json.dumps(RESPONSE_JSON) + TEMPLATE + TEMPLATE2
output_text = response["quiz"] + response["review"]
total_tokens = estimate_tokens(input_text) + estimate_tokens(output_text)

print(f"Input Tokens (estimate): {estimate_tokens(input_text)}")
print(f"Output Tokens (estimate): {estimate_tokens(output_text)}")
print(f"Total Tokens (estimate): {total_tokens}")


Input Tokens (estimate): 1006
Output Tokens (estimate): 862
Total Tokens (estimate): 1868


In [42]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems

In [45]:
quiz = response.get("quiz")

In [47]:
print(quiz)

```json
{
  "1": {
    "mcq": "Who coined the term 'machine learning'?",
    "options": {
      "a": "Donald Hebb",
      "b": "Walter Pitts",
      "c": "Arthur Samuel",
      "d": "Tom M. Mitchell"
    },
    "correct": "c"
  },
  "2": {
    "mcq": "What was the name of the experimental 'learning machine' developed by Raytheon Company in the early 1960s?",
    "options": {
      "a": "The Hebb Machine",
      "b": "Cybertron",
      "c": "The Samuel Computer",
      "d": "Nilsson's Machine"
    },
    "correct": "b"
  },
  "3": {
    "mcq": "Which of the following best describes Tom M. Mitchell's definition of machine learning?",
    "options": {
      "a": "A computer's ability to mimic human thought processes.",
      "b": "A computer program's performance improving with experience on a specific task.",
      "c": "The study of human cognitive systems.",
      "d": "Creating algorithms that mirror human thought processes."
    },
    "correct": "b"
  },
  "4": {
    "mcq": "In what